In [1]:
import pandas as pd
from src.DBInterface import DBInterface
from bson import json_util
from datetime import datetime


In [2]:
db_interface = DBInterface('mongodb://localhost:27017/?readPreference=primary', 'stocksDB', 'vietStocks')


In [3]:
def to_csv(df: pd.DataFrame) -> None:
    """
    Write a dataframe to a csv file
    :param df: A dataframe
    :return: None
    """
    today = datetime.today().strftime('%Y-%m-%d')
    df.to_csv(f'data/top50_EY_ROE_rankings_{today}.csv', index=False)


# MongoDB way

In [4]:
EYDenseRank = {
    '$setWindowFields': {"partitionBy": None, "sortBy": {"EY": -1}, "output": {"EYRank": {"$denseRank": {}}}}}

ROEDenseRank = {
    '$setWindowFields': {"partitionBy": None, "sortBy": {"roe": -1}, "output": {"ROERank": {"$denseRank": {}}}}}
match_stage = {'$match': {"$and": [{"market_cap": {"$gt": 100}}, {"pe": {"$ne": 0}}]}}
result = db_interface.collection.aggregate(
    [match_stage,
     # Create a new column, EY, which is the inverse of PE
     {'$addFields': {'EY': {'$divide': [1, '$pe']}}},
     # Create two columns: EYRank and ROERank. EYRank contains the ranking of each stock's EY, and ROERank contains the ranking of each stock's ROE 
     EYDenseRank,
     ROEDenseRank,
     {'$addFields': {'finalRank': {'$add': ['$EYRank', '$ROERank']}}},
     {'$sort': {'finalRank': 1}},
     {'$limit': 50},
     {'$project': {'_id': 0, 'symbol': 1, 'company_name': 1, 'EY': 1, 'roe': 1, 'floor': 1,
                   'EYRank': 1, 'ROERank': 1, 'finalRank': 1}}
     ]
)

# Write to a csv file
df = pd.DataFrame(result)
to_csv(df)



# Pandas way

In [5]:
# Another way to do it

all_stocks = db_interface.collection.find({})

# Write to a json file
with open('data/vietStocks.json', 'w') as f:
    f.write(json_util.dumps(all_stocks))

# Filter all stocks with eps > 0 or eps4Q > 0

stocksCollection = pd.read_json('data/vietStocks.json')
stocksCollection = stocksCollection[(stocksCollection['market_cap'] > 100) & (stocksCollection['pe'] != 0)]
stocksCollection.count()

_id                   1104
avg_trading_vol_10    1104
avg_trading_vol_5     1104
book_value            1104
company_name          1104
eps                   1104
eps4Q                 1104
ev_per_ebit           1104
ev_per_ebitda         1104
floor                 1104
icb_group             1104
id                    1104
index_code            1104
market_cap            1104
match_price_10        1104
match_price_5         1104
pb                    1104
pb4Q                  1104
pe                    1104
pe4Q                  1104
profit_after_tax      1104
roa                   1104
roe                   1104
rs1m                  1057
rs3m                  1057
rs52w                 1057
symbol                1104
the_beta              1104
the_beta4Q            1104
year                  1104
match_price           1104
change_price          1104
change_percent        1104
accumylated_vol       1104
ceiling_price         1104
floor_price           1104
date                  1104
d

In [6]:
# Create a new column, EY, which is the inverse of PE
stocksCollection['EY'] = 1 / stocksCollection['pe']

In [7]:
# Create two columns: EYRank and ROERank. EYRank contains the ranking of each stock's EY, and ROERank contains the ranking of each stock's ROE
stocksCollection['EYRank'] = stocksCollection['EY'].rank(ascending=False)
stocksCollection['ROERank'] = stocksCollection['roe'].rank(ascending=False)
stocksCollection['finalRank'] = stocksCollection['EYRank'] + stocksCollection['ROERank']

In [8]:
stocksCollection

,_id,avg_trading_vol_10,avg_trading_vol_5,book_value,company_name,eps,eps4Q,ev_per_ebit,ev_per_ebitda,floor,...,change_price,change_percent,accumylated_vol,ceiling_price,floor_price,date,EY,EYRank,ROERank,finalRank
36,{'$oid': '651034a433bdb847a5cec6d7'},0,0.0,9957,Công ty cổ phần Đầu tư Thương mại và Xuất nhập...,-543,-724,-5.60,-5.76,HNX,...,0.0,0.00,0,0.9,0.7,2023-09-27,-0.680272,1091.0,1030.0,2121.0
49,{'$oid': '651034a433bdb847a5cec6f1'},0,0.0,10415,Công ty cổ phần đầu tư và khoáng sản FLC Stone,-914,-782,-3.75,-4.98,UPCOM,...,0.0,0.00,0,1.5,0.7,2023-09-27,-0.826446,1095.0,1033.0,2128.0
65,{'$oid': '651034a433bdb847a5cec711'},0,0.0,11942,Công ty cổ phần Chứng khoán BOS,349,349,3.36,2.94,UPCOM,...,0.0,0.00,0,1.8,0.8,2023-09-27,0.268817,47.0,739.0,786.0
73,{'$oid': '651034a433bdb847a5cec721'},0,0.0,2414,Công ty Cổ phần Hùng Vương,-5619,-5707,-3.90,-4.50,UPCOM,...,0.0,0.00,0,1.6,1.2,2023-09-27,-4.000000,1104.0,1100.0,2204.0
79,{'$oid': '651034a433bdb847a5cec72d'},0,0.0,9427,Công ty Cổ phần Nông dược H.A.I,133,-3129,274.51,76.28,UPCOM,...,0.0,0.00,0,2.1,0.9,2023-09-27,0.088889,489.0,807.5,1296.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1728,{'$oid': '651034a533bdb847a5ced40f'},17114,8720.0,86685,Công ty Cổ phần Mía đường Sơn La,31557,53423,5.26,4.51,HNX,...,1.8,0.90,1150,220.1,180.1,2023-09-27,0.195312,107.5,10.0,117.5
1729,{'$oid': '651034a533bdb847a5ced411'},100,0.0,121271,Công ty Cổ phần Bia và Nước giải khát Hạ Long,41232,26691,4.28,3.08,UPCOM,...,36.2,15.00,20,277.6,205.2,2023-09-27,0.160772,169.0,23.0,192.0
1730,{'$oid': '651034a533bdb847a5ced413'},10,40.0,12992,Công ty Cổ phần Xây dựng điện VNECO4,-1851,-3123,-52.78,-81.34,HNX,...,23.7,9.95,10,262.1,214.5,2023-09-27,-0.016832,999.0,1076.0,2075.0
1731,{'$oid': '651034a533bdb847a5ced415'},10,60.0,10000,Công ty TNHH MTV Xây dựng Công trình Tân Cảng,1048,1048,438.12,300.78,UPCOM,...,0.0,0.00,0,498.5,368.5,2023-09-27,0.002418,948.0,422.0,1370.0


In [9]:
# Get the top 50 stocks with the highest finalRank
stocksCollection = stocksCollection.sort_values(by=['finalRank'], ascending=True).head(50)

In [10]:
# Get the symbol, company_name, EY, ROE, EYRank, and ROERank of the top 50 stocks

stocksCollection = stocksCollection[['symbol', 'company_name', 'EY', 'roe', 'floor', 'EYRank', 'ROERank', 'finalRank']]
today = datetime.today().strftime('%Y-%m-%d')
stocksCollection.to_csv(f'data/top50_EY_ROE_rankings_{today}.csv', index=False)
